<h1><center>Task 1</center></h1>

In [ ]:
#Importing libraries

#Data processing
import pandas as pd
import numpy as np
from numpy.random import seed
import re
import datetime as datetime
from datetime import date
import numbers

#Data visualization
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle
import seaborn as sns
import plotly.io as pio
import plotly.graph_objects as go
from mpl_toolkits.mplot3d import Axes3D

#Data modeling
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import mean_squared_error, classification_report, accuracy_score
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingRegressor, VotingClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, Normalizer, RobustScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.xgbod import XGBOD
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import VarianceThreshold
from skfeature.function.similarity_based import fisher_score
from sklearn.metrics import r2_score
from scipy.spatial.distance import mahalanobis
from scipy.stats import chi2
from sklearn.cluster import DBSCAN
from sklearn.svm import OneClassSVM
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE

#Other
import warnings
pio.renderers.default = "notebook"

In [ ]:
#Suppressing warnings
warnings.simplefilter(action = "ignore")

In [ ]:
#Setting random seed
seed(11)

In [ ]:
#Importing data
xtrain_df = pd.read_csv("X_train.csv", index_col = 0)
ytrain_df = pd.read_csv("y_train.csv", index_col = 0)
xtest_df = pd.read_csv("X_test.csv", index_col = 0)

In [ ]:
#Set feature columns
feature_columns = xtrain_df.columns.to_list()

In [ ]:
#Set params
params = {
    "random_state": 11,
    # Missing values
    "missing_values_imputation_method": "mean", #"median","mean","knn","mice"
    "mv_knn_nneighbors": 5,
    "mv_knn_weights": "distance", #"uniform","distance"
    "mv_mice_max_iter": 5,
    "mv_mice_initial_strategy": 'median', #"mean","median","most_frequent","constant"
    "mv_mice_n_nearest_features": 5,
    # Outliers
    "outlier_removal_method": "svm", #"iqr","if","md","svm","pca"
    "o_iqr_threshold":10,
    "o_if_contamination":0.1,
    "o_if_n_estimators":100,
    "o_if_max_samples": 0.9,
    "o_if_max_features": 0.9,
    "o_md_chisquarethreshold":0.95,
    "o_svm_kernel": "rbf", #"linear","poly","rbf","sigmoid","precomputed"
    "o_svm_degree": 5,
    "o_svm_nu": 0.05,
    "o_svm_gamma": "scale", #"scale","auto"
    "o_pca_n_components": 10,
    "o_pca_percentilereconstructionerror": 95,
    # Feature selection
    "feature_selection_method": "lasso", #"mi","variancethreshold","lasso","rf","rfe","pca"
    "fs_mi_threshold": 0.05,
    "fs_vt_threshold": 1.0,
    "fs_lasso_alpha": 0.4,
    "fs_rf_n_estimators": 50,
    "fs_rf_importance_threshold": 0.001,
    "fs_rfe_nfeatures": 300,
    "fs_pca_n_components": 300,
    # Scaling
    "scaling_method": "std", #"maxabs","std","minmax","robust"
    #Models
    "lasso_params": {
        "alpha": [0.]
    },
    "xgboost_params": {
        "n_estimators":[50,50],
        "learning_rate": [0.001,0.02,0.05,0.1],
        "eta": [0.05,0.3,0.5],
        "gamma": [0.5,0.68,0.85],
        "max_depth": [4,6,8],
        "subsample": [0.34,0.59,0.8],
        "colsample_bytree": [0.68, 0.8,0.9],
        "lambda": [0.65,0.85],
        "alpha": [0.4,0.2,0.5]
    },
    "randomforest_params": {
        "n_estimators": [100],
        "criterion": ["squared_error"], #“squared_error”, “absolute_error”, “friedman_mse”, “poisson”
        "max_depth": [10],
        "min_samples_split": [2],
        "max_features": [1.0]
    },
    "lgbm_params": {
        "n_estimators": [2000],
        "learning_rate": [0.02],
        "max_depth": [6],
        "subsample": [0.34],
        "colsample_bytree": [0.68],
        "reg_lambda": [0.65],
        "reg_alpha": [0.4]
    }
}

In [ ]:
#To do: Add hyperparameters

### Columns

In [ ]:
def drop_constants(X_train,X_test,feature_columns):
    constant_column_mask = xtrain_df.nunique() == 1
    constants = constant_column_mask[constant_column_mask].index.to_list()
    for constant in constants:
        X_train = X_train.drop(constant, axis = 1)
        X_test = X_test.drop(constant, axis = 1)
        feature_columns.remove(constant)
    return X_train,X_test,feature_columns

In [ ]:
xtrain_df, xtest_df, feature_columns = drop_constants(xtrain_df, xtest_df, feature_columns)

### Missing values

In [ ]:
def missing_value_imputation(X_train, X_test, params):
    if params['missing_values_imputation_method'] == 'median':
        X_train_imp = np.where(np.isnan(np.array(X_train)), np.nanmedian(np.array(X_train), axis = 0), np.array(X_train))
        X_test_imp = np.where(np.isnan(np.array(X_test)), np.nanmedian(np.array(X_test), axis = 0), np.array(X_test))
    
    elif params['missing_values_imputation_method'] == 'mean':
        X_train_imp = np.where(np.isnan(np.array(X_train)), np.nanmean(np.array(X_train), axis = 0), np.array(X_train))
        X_test_imp = np.where(np.isnan(np.array(X_test)), np.nanmean(np.array(X_test), axis = 0), np.array(X_test))
    
    elif params['missing_values_imputation_method'] == 'knn':
        imputer = KNNImputer(n_neighbors=params["mv_knn_nneighbors"], weights=params["mv_knn_weights"])
        imputed_values_train = imputer.fit_transform(X_train)
        imputed_values_test = imputer.transform(X_test)
        X_train_imp = np.where(np.isnan(X_train), imputed_values_train, X_train)
        X_test_imp = np.where(np.isnan(X_test), imputed_values_test, X_test)
    
    elif params['missing_values_imputation_method'] == 'mice':
        imputer = IterativeImputer(max_iter=params["mv_mice_max_iter"], initial_strategy=params["mv_mice_initial_strategy"],n_nearest_features=params["mv_mice_n_nearest_features"])
        imputed_values_train = imputer.fit_transform(X_train)
        imputed_values_test = imputer.transform(X_test)
        X_train_imp = np.where(np.isnan(X_train), imputed_values_train, X_train)
        X_test_imp = np.where(np.isnan(X_test), imputed_values_test, X_test)
    
    X_train = pd.DataFrame(X_train_imp, index = X_train.index, columns = X_train.columns)
    X_test = pd.DataFrame(X_test_imp, index = X_test.index, columns = X_test.columns)
        
    return X_train, X_test

In [ ]:
xtrain_df, xtest_df = missing_value_imputation(xtrain_df, xtest_df, params)

### Outliers

In [ ]:
def outlier_removal(X_train, y_train, params):
    if params['outlier_removal_method'] == 'iqr':
        q1 = xtrain_df.quantile(0.25)
        q3 = xtrain_df.quantile(0.75)
        iqr = q3 - q1
        mask = (xtrain_df < (q1 - params["o_iqr_threshold"] * iqr)) | (xtrain_df > (q3 + params["o_iqr_threshold"] * iqr))
        outlier_mask = mask.max(axis=1)
    
    elif params['outlier_removal_method'] == 'if':
        model = IForest(contamination=params["o_if_contamination"], n_estimators=params["o_if_n_estimators"], max_samples=params["o_if_max_samples"], max_features=params["o_if_max_features"])
        model.fit(xtrain_df)
        outlier_mask = model.predict(xtrain_df)
        outlier_mask = outlier_mask == 1
        
    elif params['outlier_removal_method'] == 'md':
        X_train_a = np.array(xtrain_df)

        mean_vec = np.mean(X_train_a, axis=0)
        cov_matrix = np.cov(X_train_a, rowvar=False)
        inv_cov_matrix = np.linalg.inv(cov_matrix)

        def mahalanobis_distance(x, mean_vec, inv_cov_matrix):
            diff = x - mean_vec
            return np.sqrt(np.dot(np.dot(diff, inv_cov_matrix), diff.T))

        m_distances = np.array([mahalanobis_distance(x, mean_vec, inv_cov_matrix) for x in X_train_a])
        
        threshold = np.sqrt(chi2.ppf(params["o_md_chisquarethreshold"], df=X_train.shape[1]))
        outlier_mask = m_distances > threshold
    
    elif params['outlier_removal_method'] == 'svm':
        X_train_a = np.array(xtrain_df)

        ocsvm = OneClassSVM(kernel=params["o_svm_kernel"], nu=params["o_svm_nu"], gamma=params["o_svm_gamma"], degree=params["o_svm_degree"])
        ocsvm.fit(X_train_a)
        predictions = ocsvm.predict(X_train_a)

        outlier_mask = predictions == -1
    
    elif params['outlier_removal_method'] == 'pca':
        X_train_a = np.array(xtrain_df) 

        pca = PCA(n_components=params["o_pca_n_components"])
        X_train_pca = pca.fit_transform(X_train_a)
        X_train_reconstructed = pca.inverse_transform(X_train_pca)
        reconstruction_error = np.mean((X_train - X_train_reconstructed) ** 2, axis=1)

        threshold = np.percentile(reconstruction_error, params["o_pca_percentilereconstructionerror"])
        outlier_mask = reconstruction_error > threshold

    X_train_cleaned = pd.DataFrame(X_train[~outlier_mask],index=X_train[~outlier_mask].index,columns=X_train.columns)
    y_train_cleaned = pd.DataFrame(y_train[~outlier_mask],index=y_train[~outlier_mask].index,columns=y_train.columns)
    
    n_outliers = sum(outlier_mask)
    
    return X_train_cleaned,y_train_cleaned,n_outliers

In [ ]:
xtrain_df, ytrain_df, n_outliers = outlier_removal(xtrain_df, ytrain_df, params)

### Deskewing

In [ ]:
def deskew(X_train, X_test):
    for col in X_train.columns:
        try:
            pt = PowerTransformer(method="yeo-johnson")
            deskewed_col_train = pt.fit_transform(np.array(X_train[col]).reshape(-1, 1))
            deskewed_col_test = pt.transform(np.array(X_test[col]).reshape(-1, 1))
            X_train[col] = deskewed_col_train
            X_test[col] = deskewed_col_test
        except: 
            X_train[col] = X_train[col]
            X_test[col] = X_test[col]
    return X_train, X_test

In [ ]:
xtrain_df, xtest_df = deskew(xtrain_df, xtest_df)

### Feature Selection

In [ ]:
def feature_selection(X_train, y_train, X_test, params):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
        
    if params['feature_selection_method'] == 'mi':
        mi = mutual_info_regression(X_train, y_train)
        mi_df = pd.DataFrame({'Feature': X_train.columns, 'Mutual Information': mi})
        mi_df = mi_df.sort_values(by='Mutual Information', ascending=False)
        selected_features = mi_df[mi_df['Mutual Information'] > params["fs_mi_threshold"]]['Feature']
    
    elif params['feature_selection_method'] == 'variancethreshold':
        selector = VarianceThreshold(threshold = params["fs_vt_threshold"])
        X_train_selected = selector.fit_transform(X_train)
        selected_features = X_train.columns[selector.get_support()]

    elif params['feature_selection_method'] == 'lasso':
        lasso = Lasso(alpha=params["fs_lasso_alpha"])
        lasso.fit(X_train_scaled, y_train)
        selected_features = X_train.columns[lasso.coef_ != 0]
    
    elif params['feature_selection_method'] == 'rf':
        rf = RandomForestRegressor(n_estimators = params["fs_rf_n_estimators"])
        rf.fit(X_train, y_train)
        importance = rf.feature_importances_
        feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': importance})
        feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
        threshold = params["fs_rf_importance_threshold"]
        selected_features = feature_importance_df[feature_importance_df['Importance'] > threshold]['Feature']
    
    elif params['feature_selection_method'] == 'rfe':
        model = LinearRegression()
        rfe = RFE(estimator=model, n_features_to_select=params["fs_rfe_nfeatures"])
        rfe.fit(X_train, y_train)
        selected_features = X_train.columns[rfe.support_]
    
    elif params['feature_selection_method'] == 'pca':
        pca = PCA(n_components=params["fs_pca_n_components"])
        X_train_pca = pca.fit_transform(X_train_scaled)
        X_test_pca = pca.transform(X_test_scaled)
        pca_components_train_df = pd.DataFrame(X_train_pca, columns=[f'PC{i+1}' for i in range(X_train_pca.shape[1])], index = X_train.index)
        pca_components_test_df = pd.DataFrame(X_test_pca, columns=[f'PC{i+1}' for i in range(X_test_pca.shape[1])], index = X_test.index)
        
    if params['feature_selection_method'] == 'pca':
        return pca_components_train_df, pca_components_test_df
        n_features = params["fs_pca_n_components"]
    else:
        X_train = X_train[selected_features]
        X_test = X_test[selected_features]
        n_features = len(selected_features)
        
        return X_train,X_test,n_features

In [ ]:
xtrain_df, xtest_df, n_features = feature_selection(xtrain_df, ytrain_df, xtest_df, params)

### Scaling

In [ ]:
def scaling(X_train, X_test, params):
    if params['scaling_method'] == 'maxabs':
        scaler = MaxAbsScaler()
    
    elif params['scaling_method'] == 'std':
        scaler = StandardScaler()
    
    elif params['scaling_method'] == 'minmax':
        scaler = MinMaxScaler()
    
    elif params['scaling_method'] == 'robust':
        scaler = RobustScaler()
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_train_scaled =  pd.DataFrame(data = X_train_scaled, columns = X_train.columns, index = X_train.index)
    X_test_scaled =  pd.DataFrame(data = X_test_scaled, columns = X_test.columns, index = X_test.index)
    
    return X_train_scaled, X_test_scaled

In [ ]:
xtrain_df, xtest_df = scaling(xtrain_df, xtest_df, params)

In [ ]:
X_train, X_vali, y_train, y_vali = train_test_split(xtrain_df, ytrain_df, test_size = 0.2, random_state = params["random_state"])

### Regression models

#### Linear regression

In [ ]:
def linear_regression(X_rest, y_rest, X_train, y_train, X_vali, y_vali, params):
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_vali)
    r2 = r2_score(y_vali, y_pred)

    return model, r2

In [ ]:
lir_model, lir_r2 = linear_regression(xtrain_df, ytrain_df, X_train, y_train, X_vali, y_vali, params)

#### Lasso regression

In [ ]:
def lasso_regression(X_rest, y_rest, X_train, y_train, X_vali, y_vali, params):
    model_params = params["lasso_params"]
    
    gs = GridSearchCV(linear_model.Lasso(), 
                      param_grid = model_params,
                      cv = 3)
    
    gs.fit(X_rest, y_rest)
    
    y_pred = gs.predict(X_vali)
    r2 = r2_score(y_vali, y_pred)
        
    return gs, r2, gs.best_params_

In [ ]:
lar_model, lar_r2, lar_params = lasso_regression(xtrain_df, ytrain_df, X_train, y_train, X_vali, y_vali, params)

#### XGBoost

In [ ]:
def xgboost(X_rest, y_rest, X_train, y_train, X_vali, y_vali, params):
    model_params = params["xgboost_params"]
    
    gs = GridSearchCV(xgb.XGBRegressor(), 
                      param_grid = model_params,
                      cv = 2)
    
    gs.fit(X_rest, y_rest)
    
    y_pred = gs.predict(X_vali)
    r2 = r2_score(y_vali, y_pred)
        
    return gs, r2, gs.best_params_

In [ ]:
xgb_model, xgb_r2, xgb_params = xgboost(xtrain_df, ytrain_df, X_train, y_train, X_vali, y_vali, params)

In [ ]:
xgb_r2

#### Random forest

In [ ]:
def randomforest(X_rest, y_rest, X_train, y_train, X_vali, y_vali, params):
    model_params = params["randomforest_params"]
    
    gs = GridSearchCV(RandomForestRegressor(), 
                      param_grid = model_params,
                      cv = 2)
    
    gs.fit(X_rest, y_rest)
    
    y_pred = gs.predict(X_vali)
    r2 = r2_score(y_vali, y_pred)
        
    return gs, r2, gs.best_params_

In [ ]:
rf_model, rf_r2, rf_params = randomforest(xtrain_df, ytrain_df, X_train, y_train, X_vali, y_vali, params)

#### LightGBM

In [ ]:
def lgbm(X_rest, y_rest, X_train, y_train, X_vali, y_vali, params):
    model_params = params["lgbm_params"]
    
    gs = GridSearchCV(LGBMRegressor(), 
                      param_grid = model_params,
                      cv = 2)
    
    gs.fit(X_rest, y_rest)
    
    y_pred = gs.predict(X_vali)
    r2 = r2_score(y_vali, y_pred)
        
    return gs, r2, gs.best_params_

In [ ]:
lgbm_model, lgbm_r2, lgbm_params = lgbm(xtrain_df, ytrain_df, X_train, y_train, X_vali, y_vali, params)

#### Voting ensemble

In [ ]:
def ensemble(lir_model, lar_model, lgbm_model, xgb_model, rf_model, X_rest, y_rest, X_train, y_train, X_vali, y_vali, params):
    voting = VotingRegressor(estimators = [
        ("lir_model", lir_model),
        ("lar_model", lar_model),
        ("lgbm_model", lgbm_model),
        ("xgb_model", xgb_model),
        ("rf_model", rf_model)])
    
    voting.fit(X_train, y_train)
    
    y_pred = voting.predict(X_vali)
    r2 = r2_score(y_vali, y_pred)
        
    return voting, r2

In [ ]:
voting_model, voting_r2 = ensemble(lir_model, lar_model, lgbm_model, xgb_model, rf_model, xtrain_df, ytrain_df, X_train, y_train, X_vali, y_vali, params)

In [ ]:
print(lir_r2)
print(lar_r2)
print(xgb_r2)
print(rf_r2)
print(lgbm_r2)
print(voting_r2)

In [ ]:
y_pred = voting_model.predict(xtest_df)
sub = pd.DataFrame(data=y_pred,index=xtest_df.index)
sub.columns = ["y"]
sub.to_csv("submission.csv")

In [ ]:
y_pred = voting_model.predict(xtest_df)
sub = pd.DataFrame(data=y_pred,index=xtest_df.index)
sub.columns = ["y"]
sub.to_csv("submission.csv")